In [ ]:
#face detection

In [6]:
import numpy as np 
import cv2 
import os 
#os.chdir(r"faceeyedetection")
cap =cv2.VideoCapture('faces.mp4')

face_cascade =cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
eyes_cascade =cv2.CascadeClassifier("haarcascade_eye.xml")

while(cap.isOpened()):
    ret ,frame =cap.read()
    
    gray =cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    faces =face_cascade.detectMultiScale(gray ,1.3 , 4)
    for (x,y,h,w) in faces :
        cv2.rectangle(frame ,(x,y),(x+w ,y+h),(0,255,0),3)       
        roi_gray=gray[y:y+h ,x:x+w]       
        roi_color=frame[y:y+h ,x:x+w]       
        eyes =eyes_cascade.detectMultiScale(roi_gray)       
        for(ex,ey,eh,ew) in eyes :      
            cv2.rectangle(roi_color ,(ex,ey),(ex+ew ,ey+eh),(0,0,255),3)            
  
    
    cv2.imshow("frame" ,frame)
    
    key =cv2.waitKey(1)
    if(key == ord('q')):
        break 
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
#Object Detection

In [4]:
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# =========================
# Visualization function
# =========================
def visualize(image, detection_result):
    for detection in detection_result.detections:
        bbox = detection.bounding_box
        start_point = (bbox.origin_x, bbox.origin_y)
        end_point = (bbox.origin_x + bbox.width, bbox.origin_y + bbox.height)
        cv2.rectangle(image, start_point, end_point, (0, 255, 0), 2)

        # Draw label
        category = detection.categories[0]
        category_name = category.category_name
        score = round(category.score, 2)
        result_text = f'{category_name} ({score})'
        cv2.putText(image, result_text, (bbox.origin_x, bbox.origin_y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    return image

# =========================
# Load MediaPipe EfficientDet model
# =========================
model_path = 'efficientdet_lite0.tflite'  # Ensure this file exists in the working directory

base_options = python.BaseOptions(model_asset_path=model_path)
options = vision.ObjectDetectorOptions(base_options=base_options, score_threshold=0.5)
detector = vision.ObjectDetector.create_from_options(options)

# =========================
# Real-time webcam detection
# =========================
cap = cv2.VideoCapture('faces.mp4')  # Use camera

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    # Convert BGR to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Convert OpenCV image to MediaPipe Image
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

    # Perform detection
    detection_result = detector.detect(mp_image)

    # Visualize detection results
    frame = visualize(frame, detection_result)

    # Show the frameq
    cv2.imshow("MediaPipe Object Detection", frame)

    # Break with 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
